In [32]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio
import xarray as xr

In [33]:
# 
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"
cwd_Images_Results = cwd + "\\Results"
cwd_PRISMA = cwd + "\\PRISMA"

In [34]:
# xr.set_options(display_style='text')

# Read data

In [35]:
data_Coordinates = xr.open_dataset(cwd_PRISMA + "\\FLX_L2__FLXSYN_yyyymmddThhmmss_yyyymmddThhmmss_YYYYMMDDTHHMMSS_CCC_LLL_BB_01.nc", group = "Ancillary_Information" ,engine="netcdf4")
data_Coordinates['latitude'].shape
data_Coordinates

<xarray.Dataset> Size: 80kB
Dimensions:    (number_of_across_track_samples: 100,
                number_of_along_track_samples: 100)
Dimensions without coordinates: number_of_across_track_samples,
                                number_of_along_track_samples
Data variables:
    latitude   (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...
    longitude  (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...

In [36]:
data_Fluo = xr.open_dataset(cwd_PRISMA + "\\FLX_L2__FLXSYN_yyyymmddThhmmss_yyyymmddThhmmss_YYYYMMDDTHHMMSS_CCC_LLL_BB_01.nc", group = "L2_Fluorescence" ,engine="netcdf4")
data_Fluo

<xarray.Dataset> Size: 23MB
Dimensions:                              (
                                          number_of_real_reflectance_spectral_samples: 141,
                                          number_of_across_track_samples: 100,
                                          number_of_along_track_samples: 100,
                                          number_of_sif_spectral_samples: 56,
                                          number_of_sif_peaks: 2,
                                          number_of_sif_o2_values: 2)
Dimensions without coordinates: number_of_real_reflectance_spectral_samples,
                                number_of_across_track_samples,
                                number_of_along_track_samples,
                                number_of_sif_spectral_samples,
                                number_of_sif_peaks, number_of_sif_o2_values
Data variables:
    floris_real_reflectance              (number_of_real_reflectance_spectral_samples, number_of_across_track_samples, number_of_along_track_samples) float64 11MB ...
    floris_real_reflectance_uncertainty  (number_of_real_reflectance_spectral_samples, number_of_across_track_samples, number_of_along_track_samples) float32 6MB ...
    reflectance_wavelength_grid          (number_of_real_reflectance_spectral_samples) float32 564B ...
    sif_emission_spectrum                (number_of_sif_spectral_samples, number_of_across_track_samples, number_of_along_track_samples) float64 4MB ...
    sif_emission_spectrum_uncertainty    (number_of_sif_spectral_samples, number_of_across_track_samples, number_of_along_track_samples) int16 1MB ...
    sif_wavelength_grid                  (number_of_sif_spectral_samples) float32 224B ...
    total_integrated_sif                 (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...
    sif_peak_values                      (number_of_sif_peaks, number_of_across_track_samples, number_of_along_track_samples) float32 80kB ...
    sif_peak_positions                   (number_of_sif_peaks, number_of_across_track_samples, number_of_along_track_samples) float32 80kB ...
    sif_O2_bands_value                   (number_of_sif_o2_values, number_of_across_track_samples, number_of_along_track_samples) float32 80kB ...

In [37]:
data_Veg = xr.open_dataset(cwd_PRISMA + "\\FLX_L2__FLXSYN_yyyymmddThhmmss_yyyymmddThhmmss_YYYYMMDDTHHMMSS_CCC_LLL_BB_01.nc", group = "L2_Vegetation" ,engine="netcdf4")
data_Veg

<xarray.Dataset> Size: 120kB
Dimensions:                   (number_of_across_track_samples: 100,
                               number_of_along_track_samples: 100)
Dimensions without coordinates: number_of_across_track_samples,
                                number_of_along_track_samples
Data variables:
    leaf_area_index           (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...
    leaf_chlorophyll_content  (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...
    leaf_carotenoid_content   (number_of_across_track_samples, number_of_along_track_samples) float32 40kB ...

# ROI 3X3

In [38]:
data_Fluo['sif_emission_spectrum']

<xarray.DataArray 'sif_emission_spectrum' (number_of_sif_spectral_samples: 56,
                                           number_of_across_track_samples: 100,
                                           number_of_along_track_samples: 100)> Size: 4MB
[560000 values with dtype=float64]
Dimensions without coordinates: number_of_sif_spectral_samples,
                                number_of_across_track_samples,
                                number_of_along_track_samples
Attributes:
    standard_name:  sif_emission_spectrum
    short_name:     sif_spectrum
    units:          mW.m-2.sr-1.nm-1
    long_name:      Solar Induced Fluorescence emission spectrum
    description:    Solar Induced Fluorescence emission

In [40]:
data_Coordinates['latitude']

<xarray.DataArray 'latitude' (number_of_across_track_samples: 100,
                              number_of_along_track_samples: 100)> Size: 40kB
array([[12.52469 , 12.523891, 12.523093, ..., 12.44806 , 12.447242, 12.446423],
       [12.521181, 12.520383, 12.519585, ..., 12.444563, 12.443745, 12.442926],
       [12.517673, 12.516874, 12.516076, ..., 12.441065, 12.440248, 12.439429],
       ...,
       [12.176602, 12.175805, 12.175008, ..., 12.101213, 12.100413, 12.099615],
       [12.172974, 12.172176, 12.171379, ..., 12.097595, 12.096796, 12.095997],
       [12.169346, 12.168548, 12.167749, ..., 12.093978, 12.093179, 12.09238 ]],
      dtype=float32)
Dimensions without coordinates: number_of_across_track_samples,
                                number_of_along_track_samples
Attributes:
    standard_name:  latitude
    short_name:     lat
    units:          degrees_north
    long_name:      Latitude
    description:    Latitude ground coordinate associated to each pixel in WG...

In [41]:
data_Coordinates['longitude']

<xarray.DataArray 'longitude' (number_of_across_track_samples: 100,
                               number_of_along_track_samples: 100)> Size: 40kB
[10000 values with dtype=float32]
Dimensions without coordinates: number_of_across_track_samples,
                                number_of_along_track_samples
Attributes:
    standard_name:  longitude
    short_name:     lon
    units:          degrees_east
    long_name:      Longitude
    description:    Longitude ground coordinate associated to each pixel in W...